In [4]:
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import time
# own code
import libarary.graph_tools as gtl
import libarary.invarients as inva
import libarary.cpp as cpp

In [5]:
cpp.test()

hello world


In [6]:
G = gtl.read_graph_from_edgelist("networks-random/brain274/bn-mouse_brain_1.edges")


In [7]:
start = time()
res = inva.q_extent_single(3,G)
end = time()
clock = end-start
print("time: ",clock)
print("results: ",res)

time:  2.630380153656006
results:  0.0001771636105943839


In [8]:
start = time()


dists = gtl.convert_dist_dict_to_narray(dict(nx.all_pairs_dijkstra_path_length(G)),list(G.nodes))
# cpp.q_extend_single()
end = time()
clock  = end -start
print(clock)

0.547663688659668


In [9]:
G = gtl.read_graph_from_edgelist(
        "networks-random/memalia-big/mammalia-voles-bhp-trapping.edges"
    )
start = time()
dists = gtl.convert_dist_dict_to_narray(dict(nx.all_pairs_dijkstra_path_length(G)),list(G.nodes))
res= cpp.q_extend_single(dists, 3)
end = time()
print("results: ", res)
print("time: ", end-start)


results:  1.4079995494401442e-06
time:  2.5852622985839844
